# PROGETTO DWM 2021
### Giovanni Costa - 880892

Indice:
- [Analisi del dataset "Train"](#analisi_train)
- [Analisi del dataset "Properties"](#analisi_prop)
- [Features Engineering](#features_engineer)
    - [Gestione dei missing values e rimozione delle colonne non necessarie o che presentano multicollinearità](#missing_val)
        - [Recupero di missing values per features connesse a posizione geografica e tassazione](#recover_missing_pos_tax)
    - [Aggiunta di features custom potenzialmente utili](#custom_features) 
- [Features importance, features selection e preparazione del dataset finale](#features_selection)
- [Corstruzione del modello sfruttante Random Forest e tuning dei suoi parametri](#mod_1)
- [Corstruzione del modello sfruttante Gradient Boosting e tuning dei suoi parametri](#mod_2)
- [Comparazione e analisi dei modelli](#comparison_and_analisys)
- [Considerazioni finali](#final)

In [ ]:
# Library imports
%matplotlib inline
%load_ext autoreload
%autoreload 2

from utils.general_utils import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings, random, joblib, os
import xgboost as xgb
from sklearn.utils import resample
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
from sklearn.feature_selection import RFECV
from sklearn.model_selection import GridSearchCV

# Set up the working directory
INPUT_DATA_DIR = "../data/input"
OUTPUT_DATA_DIR = "../data/output"
MODEL_DIR = "models"
PROPS_FILENAME = "properties_2016.parquet"
TRAIN_DATA_FILENAME = "train_2016_v2.csv"
PROPS_PATH = os.path.join(INPUT_DATA_DIR, PROPS_FILENAME)
TRAIN_DATA_PATH = os.path.join(INPUT_DATA_DIR, TRAIN_DATA_FILENAME)
SEED = 42
TOP_K_DISPLAY = 20

# Set up the random seed
np.random.seed(SEED)
random.seed(SEED)
# Display option
pd.options.display.float_format = '{:.4f}'.format

<a id='analisi_train'></a>
### Analisi del dataset "Train"

In [ ]:
df_train = pd.read_csv(
    TRAIN_DATA_PATH, parse_dates=["transactiondate"], date_format="%Y-%m-%d"
)
df_train[df_train.select_dtypes(np.float64).columns] = df_train.select_dtypes(
    np.float64
).astype(np.float32)
df_train.info(max_cols=TOP_K_DISPLAY)
print("Shape: ", df_train.shape)

In [ ]:
# Missing values ratio
df_train.isna().sum() / df_train.shape[0] * 100

Notiamo che il dataset contiene 3 colonne:
- Parcelid: id univoco che identifica ogni istanza di edificio
- Logerror: indice che dovremmo utilizzare per verificare la bontà del nostro modello<br>
Dal sito internet della competizione: *"Zillow is asking you to predict the log-error between their Zestimate and the actual sale price, given all the features of a home. The log error is defined as logerror=log(Zestimate)−log(SalePrice)"*
- Transactiondate: data di vendita reale o stimata per quell'istanza di edificio

Inoltre non ci sono valori nulli nel dataset ma ci sono parcelid duplicati, anche se la coppia parcelid e transactiondate è univoca per ogni riga. Questo implica che ci sono dati di vendite reali (o previste) con riferimento allo stesso edificio in diverse giornate

<a id='analisi_prop'></a>
### Analisi del dataset "Properties"

In [ ]:
df_prop = pd.DataFrame([])
if PROPS_PATH.endswith(".csv"):
    df_prop = pd.read_csv(PROPS_PATH)
    to_float64_float32(df_prop)
    df_prop.to_parquet(
        os.path.join(INPUT_DATA_DIR, PROPS_FILENAME.split(".")[0] + ".parquet")
    )
else:
    df_prop = pd.read_parquet(PROPS_PATH)

# df_prop=pd.read_csv(input_folder+data_file_name, header=0) #parametro nrows=x per limitare il numero di righe

df_prop.info(max_cols=TOP_K_DISPLAY)
print("Shape: ", df_prop.shape)

In [ ]:
# Missing values ratio
df_prop.isna().sum().sort_values(ascending=False)[:TOP_K_DISPLAY] / df_prop.shape[
    0
] * 100

In [ ]:
# Missing values plot
df_missing = df_prop.isnull().sum(axis=0).reset_index()
df_missing.columns = ["column_name", "missing_count"]
df_missing = df_missing.loc[df_missing["missing_count"] > 0]
df_missing = df_missing.sort_values(by="missing_count")

ind = np.arange(df_missing.shape[0])
fig, ax = plt.subplots(figsize=(12, 18))
rects = ax.barh(ind, df_missing["missing_count"])
ax.set_yticks(ind)
ax.set_yticklabels(df_missing["column_name"], rotation="horizontal")
ax.set_xlabel("Count of missing values")
ax.set_title("Number of missing values in each column")
plt.show()
del df_missing

In [114]:
# Colonne contenti variabili categoriali sulla base della ducumentazione fornita su kaggle.com (usate successivamente)
other_cols = [
    "parcelid",
    "airconditioningtypeid",
    "architecturalstyletypeid",
    "buildingqualitytypeid",
    "buildingclasstypeid",
    "decktypeid",
    "fips",
    "fireplaceflag",
    "hashottuborspa",
    "heatingorsystemtypeid",
    "pooltypeid10",
    "pooltypeid2",
    "pooltypeid7",
    "propertycountylandusecode",
    "propertylandusetypeid",
    "propertyzoningdesc",
    "rawcensustractandblock",
    "censustractandblock",
    "regionidcounty",
    "regionidcity",
    "regionidzip",
    "regionidneighborhood",
    "regionidzip",
    "storytypeid",
    "typeconstructiontypeid",
    "assessmentyear",
    "taxdelinquencyflag",
    "taxdelinquencyyear",
    "yearbuilt",
]
numerical_cols = [x for x in df_prop.columns if x not in other_cols]

In [ ]:
# Visualizzazione dei valori delle colonne di tipo non float, int o datetime
not_float_col = df_prop.select_dtypes(exclude=[np.float32, np.int64]).columns
for c in not_float_col:
    print("Column: " + c)
    print("values: ", df_prop[c].unique()[:TOP_K_DISPLAY], "\n")

Il dataset presenta molte features e istanze:
- alcune di queste aventi tipo float32 sono in realtà identificativi numerici di tipo intero associati a categorie descritte più approfonditamente nella documentazione di Kaggle.com
- quelle di tipo object invece sono stringhe rappresentanti valori booleani oppure codici relativi a categorie

Le varie colonne verranno quindi gestite successivamente per rappresentare la corretta semantica del dato.

Il dataset inoltre non presenta duplicati né in parcelid né globalmente, ma contiene un numero molto elevato di valori mancanti: diverse colonne infatti presentano un missing ratio superiore al 97%. Nei capitoli successivi verrà illustrato come vengono gestiti tali valori mancanti per ognuna delle features presenti

Per questo particolare task di supervised learning sono sufficienti solamente le abitazioni all'interno del file "train_2016_v2" e non tutte quelle presenti in "properties_2016", si procede quindi all'unione dei due dataset analizzati precedentemente in modo da averne uno unico contenente l'insieme di tutte le colonne e solamente le righe che verranno effettivamente utilizzate per la previsione

In [ ]:
df_prop.info(max_cols=TOP_K_DISPLAY)

Dataset dopo il merge:

In [ ]:
df_prop = df_train.merge(df_prop, on="parcelid")
df_prop.info(max_cols=TOP_K_DISPLAY)
print("Shape: ", df_prop.shape)

<a id='features_engineer'></a>
## Features Engineering

<a id='missing_val'></a>
### Gestione dei missing values e rimozione delle colonne non necessarie o che presentano multicollinearità

In [ ]:
# Heatmap per visualizzare le correlazioni tra le variabili
plt.figure(figsize=(12, 8))
sns.heatmap(data=df_prop[numerical_cols].corr(), cmap="Reds")
plt.show()
plt.gcf().clear()

Dall'analisi della heatmap sulla correlazione tra le variabili numeriche si nota che le features 'calculatedfinishedsquarefeet', 'finishedsquarefeet12', 'finishedsquarefeet13', 'finishedsquarefeet15' e 'finishedsquarefeet6' sono molto correlate essendo di colore rosso scuro.
La stessa considerazione si applica anche a 'finishedfloor1squarefeet' e 'finishedsquarefeet50' che in aggiunta nella documentazione fornita hanno la stessa descrizione.
Similmente anche 'bathroomcnt', 'calculatedbathnbr' e 'fullbathcnt' sono correlate

Viene mantenuta solamente 'calculatedfinishedsquarefeet' tra le quattro perché presenta meno valori missing, mentre tra 'finishedfloor1squarefeet' e 'finishedsquarefeet50' viene rimossa arbitrariamente 'finishedsquarefeet50' in quanto il missing ratio è equivalente.

Vengono eliminati anche 'bathroomcnt' e 'calculatedbathnbr' lasciando 'bathroomcnt', in quanto viene applicata un'intuizione analoga alla precedente

In [ ]:
df_prop[
    [
        "calculatedfinishedsquarefeet",
        "finishedsquarefeet12",
        "finishedsquarefeet13",
        "finishedsquarefeet15",
        "finishedsquarefeet6",
    ]
].isna().sum() / df_prop.shape[0] * 100

In [120]:
df_prop.drop(
    columns=[
        "finishedsquarefeet12",
        "finishedsquarefeet13",
        "finishedsquarefeet15",
        "finishedsquarefeet6",
    ],
    inplace=True,
)

In [ ]:
df_prop[
    ["finishedfloor1squarefeet", "finishedsquarefeet50"]
].isna().sum() / df_prop.shape[0] * 100

In [122]:
df_prop.drop(columns="finishedsquarefeet50", inplace=True)

In [ ]:
df_prop[
    ["calculatedbathnbr", "bathroomcnt", "fullbathcnt"]
].isna().sum() / df_prop.shape[0] * 100

In [124]:
df_prop.drop(columns=["calculatedbathnbr", "fullbathcnt"], inplace=True)

'hashottuborspa' e 'pooltypeid10' presentano semanticamente la stessa descrizione. Si decide di rimuovere 'pooltypeid10' che presenta meno valori missing.

Viene inoltre assunto che se il valore di pool/hot tub (features 'pooltypeid2', 'pooltypeid7', 'poolcnt') non è presente questo indichi 0 elementi presenti nell'edificio

In [ ]:
print(df_prop["hashottuborspa"].value_counts())
print(df_prop["pooltypeid10"].value_counts())

In [ ]:
df_prop[["hashottuborspa", "pooltypeid10"]].isnull().sum() / df_prop.shape[0] * 100

In [127]:
df_prop.drop(columns="pooltypeid10", inplace=True)

In [128]:
df_prop[["pooltypeid2", "pooltypeid7", "poolcnt"]] = df_prop[
    ["pooltypeid2", "pooltypeid7", "poolcnt"]
].fillna(0)
df_prop["hashottuborspa"] = df_prop["hashottuborspa"].fillna(0)  # >90% nan
df_prop["hashottuborspa"] = df_prop["hashottuborspa"].astype(bool);

Per quanto riguarda la features 'poolsizesum' viene utilizzata la mediana dei valori presenti alle righe in cui la colonna 'poolcnt' è meggiore di 0, altrimenti questo valore viene impostato a 0
(si sceglie di usare la mediana come filler perchè meno influenzata dagli outliers e perchè si assume che le dimensioni delle piscine negli Stati Uniti siano più o meno standard) 

In [129]:
median_poolsize = df_prop[df_prop["poolcnt"] > 0]["poolsizesum"].median()
df_prop.loc[
    (df_prop["poolcnt"] > 0) & (df_prop["poolsizesum"].isna()), "poolsizesum"
] = median_poolsize

# Se non ha la piscina la dimensione della piscina è 0
df_prop.loc[(df_prop["poolcnt"] == 0), "poolsizesum"] = 0

'fireplaceflag' e 'fireplacecnt' presentano delle inconsistenze:
come si nota dall'analisi sottostante ci sono righe dove 'fireplacecnt' è presente, mentre 'fireplaceflag' è missing oppure errato. 

Si prosegue dunque mettendo a 0 i valori di 'fireplacecnt' quando NaN e impostando a True 'fireplaceflag' quando 'fireplacecnt' è presente, False altrimenti

In [130]:
df_prop["fireplaceflag"] = df_prop["fireplaceflag"].fillna(0)  # >90% nan
df_prop["fireplaceflag"] = df_prop["fireplaceflag"].astype(bool)
df_prop.loc[(~df_prop["fireplacecnt"].isna()), "fireplaceflag"] = True
df_prop["fireplacecnt"] = df_prop["fireplacecnt"].fillna(0)

Vengono riempiti i valori di 'taxdelinquencyflag', 'garagecarcnt', 'garagetotalsqft' assumendo 0 se il valore è NaN, come fatto in precedenza

In [131]:
df_prop[["taxdelinquencyflag", "garagecarcnt", "garagetotalsqft"]] = df_prop[
    ["taxdelinquencyflag", "garagecarcnt", "garagetotalsqft"]
].fillna(0)
df_prop["taxdelinquencyflag"] = df_prop["taxdelinquencyflag"].astype(bool)

Le features come 'airconditioningtypeid', 'heatingorsystemtypeid',  'threequarterbathnbr' invece si ritengono poco importanti e variabili così si decide di rimpiazzare i valori missing di tali features categoriali con la loro rispettiva moda.<br>
Come si nota dai grafici sottastanti la frequenza di queste variabili categoriali è intuitivamente corretta, ragionevolmente si ipotizza infatti che AC e Heating System siano più comunemente 'Central' e che la maggior parte delle abitazioni abbiano solamente un bagno a trequarti

In [ ]:
plt.figure(figsize=(8, 5))
df_prop["airconditioningtypeid"].astype(int, errors="ignore").hist(grid=False)
plt.show()

mode = float(df_prop["airconditioningtypeid"].mode()[0])
print("Moda: ", mode)
df_prop["airconditioningtypeid"] = df_prop["airconditioningtypeid"].fillna(mode)

In [ ]:
plt.figure(figsize=(10, 6))
df_prop["heatingorsystemtypeid"].astype(int, errors="ignore").hist(grid=False)
plt.show()

mode = float(df_prop["heatingorsystemtypeid"].mode()[0])
print("Moda: ", mode)
df_prop["heatingorsystemtypeid"] = df_prop["heatingorsystemtypeid"].fillna(mode)

In [ ]:
plt.figure(figsize=(5, 5))
df_prop["threequarterbathnbr"].astype(int, errors="ignore").hist(grid=False)
plt.show()

mode = float(df_prop["threequarterbathnbr"].mode()[0])
print("Moda: ", mode)
df_prop["threequarterbathnbr"] = df_prop["threequarterbathnbr"].fillna(mode)

Si decide infine ora di rimuovere le features con missing ratio maggiore di 97% perché ritenute con troppa poca informazione per essere utili al task di regressione

In [135]:
tmp_col = df_prop.columns
for c in tmp_col:
    if df_prop[c].isna().sum() / df_prop.shape[0] > 0.97:
        df_prop.drop(columns=c, inplace=True)

In [ ]:
tmp_col = df_prop.columns
tmp_list = []

for c in tmp_col:
    if df_prop[c].isna().sum() / df_prop.shape[0] > 0.0:
        tmp_list.append(c)

print("Number of features with missing values: ", len(tmp_list))
print(tmp_list)
print("Number of total features after droping: ", len(df_prop.columns))

Si analizzano quindi le features rimanenti rimanenti cercando di ripristinare gli ultimi missing values

<a id='recover_missing_pos_tax'></a>
### Recupero di missing values per features connesse a posizione geografica e tassazione


In [ ]:
geo_col_names = [
    "latitude",
    "longitude",
    "buildingqualitytypeid",
    "propertycountylandusecode",
    "propertyzoningdesc",
    "regionidcity",
    "regionidneighborhood",
    "regionidzip",
    "unitcnt",
    "yearbuilt",
]
df_geo=df_prop[geo_col_names]
df_geo.isna().sum() / df_geo.shape[0] * 100

Intuitivamente, vista inoltre l'assenza di valori mancanti, si potrebbero utilizzare le features 'latitude' e 'longitude' relativi alla posizione dell'edificio per recuperare altri attributi non presenti: abitazioni geograficamente vicine infatti si pensa che possano avere determinate caratteristiche simili<br><br>
Si ripristinano quindi i valori originali di queste due features perché come descritto nella documentazine fornita quelli all'interno del dataset sono moltiplicati per 10^6.<br>

In [138]:
df_prop["latitude"] = df_prop["latitude"] / (10**6)
df_prop["longitude"] = df_prop["longitude"] / (10**6)
# df_prop.dropna( axis = 0, subset = ['latitude', 'longitude'], inplace = True )

Si è deciso di utilizzare l'algoritmo K-nearest neighbors (KNN) per svolgere il task di recupero perché ritenuto il più adatto visto che si basa sull'apprendimento mediante analogie tra istanze vicine e vista la sua efficienza computazionale. Nella cella di codice sottostante sono inoltre spiegate le motivazioni e le ipotesi usate per il ripristino delle variabili categoriali e numeriche

In [ ]:
df_prop["buildingqualitytypeid"]

In [ ]:
warnings.simplefilter(action='ignore', category=UserWarning)
parameters = {"n_neighbors": [1, 2, 3, 4, 5, 8, 10]}


# Si ipotizza che blocchi di case vicine siano state costruite più o meno tutte nello stesso periodo e che quindi abbiano qualità simile
fillna_knn(
    df=df_prop,
    base=["latitude", "longitude"],
    target="buildingqualitytypeid",
    tuning_params=parameters,
)

# Abitazioni vicine hanno lo stesso countrylandusecode
tmp_label_enc = zoningcode2int(df=df_prop, target="propertycountylandusecode")
fillna_knn(
    df=df_prop,
    base=["latitude", "longitude"],
    target="propertycountylandusecode",
    tuning_params=parameters,
)
df_prop["propertycountylandusecode"] = tmp_label_enc.inverse_transform(
    df_prop["propertycountylandusecode"].astype(int)
)


# Abitazioni vicine hanno la stessa zoning description
tmp_label_enc = zoningcode2int(df=df_prop, target="propertyzoningdesc")
fillna_knn(
    df=df_prop,
    base=["latitude", "longitude"],
    target="propertyzoningdesc",
    tuning_params=parameters,
)
df_prop["propertyzoningdesc"] = tmp_label_enc.inverse_transform(
    df_prop["propertyzoningdesc"].astype(int)
)


# Anche in questo caso proprietà vicine si assume abbiano gli stessi regionidcity, regionidneighborhood e regionidzip
fillna_knn(
    df=df_prop,
    base=["latitude", "longitude"],
    target="regionidcity",
    tuning_params=parameters,
)

fillna_knn(
    df=df_prop,
    base=["latitude", "longitude"],
    target="regionidneighborhood",
    tuning_params=parameters,
)

fillna_knn(
    df=df_prop,
    base=["latitude", "longitude"],
    target="regionidzip",
    tuning_params=parameters,
)


# Stessa intuizione per i campi 'unitcnt' (Number of units the structure is built into), 'yearbuilt' e 'lotsizesquarefeet'
fillna_knn(
    df=df_prop,
    base=["latitude", "longitude"],
    target="unitcnt",
    tuning_params=parameters,
)

fillna_knn(
    df=df_prop,
    base=["latitude", "longitude"],
    target="yearbuilt",
    tuning_params=parameters,
)

fillna_knn_reg(
    df=df_prop,
    base=["latitude", "longitude"],
    target="lotsizesquarefeet",
    tuning_params=parameters,
)

warnings.simplefilter(action='default', category=UserWarning)

Per quanto riguarda la feature 'finishedfloor1squarefeet', come si può anche notare dalla heatmap all'inizio della sezione corrente, questa è correlata con 'calculatedfinishedsquarefeet'. Si prova quindi ad utilizzare quest'ultima per svolgerne il filling dei valori

In [ ]:
plt.figure(figsize=(13, 13))
sns.jointplot(
    x=df_prop["finishedfloor1squarefeet"].values,
    y=df_prop["calculatedfinishedsquarefeet"].values,
)
plt.ylabel("calculatedfinishedsquarefeet", fontsize=10)
plt.xlabel("finishedfloor1squarefeet", fontsize=10)
plt.title("finishedfloor1squarefeet Vs calculatedfinishedsquarefeet", fontsize=13)
plt.show()

Dal grafico si nota che in alcune abitazioni i valori delle features sono esattamente gli stessi: probabilmente alcune case hanno la loro area complessiva occupata da solamente una stanza (come potrebbe essere una sorta di studio). Vengono quindi assunte queste informazioni per vere e si procede al riempimento.<br>
Inoltre alcune righe del dataset contengono valori di 'finishedfloor1squarefeet' maggiori della dimensione totale dell'abitazione, probabilmente dovuto ad un inserimento del dato in input errato. Si decide quindi di rimuovere tali righe dal dataset

In [142]:
df_prop.loc[
    (df_prop["finishedfloor1squarefeet"].isna()) & (df_prop["numberofstories"] == 1),
    "finishedfloor1squarefeet",
] = df_prop.loc[
    (df_prop["finishedfloor1squarefeet"].isna()) & (df_prop["numberofstories"] == 1),
    "calculatedfinishedsquarefeet",
]

droprows = df_prop.loc[
    df_prop["calculatedfinishedsquarefeet"] < df_prop["finishedfloor1squarefeet"]
].index
df_prop = df_prop.drop(droprows)

In [ ]:
tmp_col = df_prop.columns
tmp_list = []

for c in tmp_col:
    if df_prop[c].isna().sum() / df_prop.shape[0] > 0.0:
        tmp_list.append(c)

print("Number of features with missing values: ", len(tmp_list))
print(tmp_list)

Si gestiscono ora le variabili inerenti alle tasse sugli edifici: in particolare si prova a recuperare i valori di 'structuretaxvaluedollarcnt', 'taxamount' e 'landtaxvaluedollarcnt'.
La variabile 'taxvaluedollarcnt' si ipotizza essere la più significativa da usare come supporto in quanto contenente anche minor numero di valori missing.<br>

Viene svolto quindi il filling dei valori NaN di 'taxvaluedollarcnt' usando la sua mediana, in modo da avere un risultato meno sensibile agli outliers.

In [ ]:
tax_col_names = [
    "taxvaluedollarcnt",
    "landtaxvaluedollarcnt",
    "structuretaxvaluedollarcnt",
    "taxamount",
]
df_tax = df_prop[tax_col_names]
df_tax.isna().sum() / df_tax.shape[0] * 100

In [145]:
median = df_prop["taxvaluedollarcnt"].median()
df_prop["taxvaluedollarcnt"] = df_prop["taxvaluedollarcnt"].fillna(median)

Da analisi su correlazione e grafici delle distribuzioni per le tre variabili target si nota inoltre che 'taxvaluedollarcnt' è la variabile più correlata per tutte: si prova quindi a svolgere una predizione dei valori missing usando l'algoritmo KNN per le features 'structuretaxvaluedollarcnt', 'taxamount' e 'landtaxvaluedollarcnt'

In [ ]:
x = df_tax.corr()

print("Target: structuretaxvaluedollarcnt")
print(x["structuretaxvaluedollarcnt"].sort_values(ascending=False))
plt.figure(figsize=(12, 12))
sns.jointplot(
    x=df_tax["structuretaxvaluedollarcnt"].values, y=df_tax["taxvaluedollarcnt"].values
)
plt.ylabel("taxvaluedollarcnt", fontsize=12)
plt.xlabel("structuretaxvaluedollarcnt", fontsize=12)
plt.title("structuretaxvaluedollarcnt Vs taxvaluedollarcnt", fontsize=15)
plt.show()

print("Target: taxamount")
print(x["taxamount"].sort_values(ascending=False))
plt.figure(figsize=(12, 12))
sns.jointplot(x=df_tax["taxamount"].values, y=df_tax["taxvaluedollarcnt"].values)
plt.ylabel("taxvaluedollarcnt", fontsize=12)
plt.xlabel("taxamount", fontsize=12)
plt.title("taxamount Vs taxvaluedollarcnt", fontsize=15)
plt.show()

print("Target: landtaxvaluedollarcnt")
print(x["landtaxvaluedollarcnt"].sort_values(ascending=False))
plt.figure(figsize=(12, 12))
sns.jointplot(
    x=df_tax["landtaxvaluedollarcnt"].values, y=df_tax["taxvaluedollarcnt"].values
)
plt.ylabel("taxvaluedollarcnt", fontsize=12)
plt.xlabel("landtaxvaluedollarcnt", fontsize=12)
plt.title("landtaxvaluedollarcnt Vs taxvaluedollarcnt", fontsize=15)
plt.show()

In [ ]:
parameters = {"n_neighbors": [10, 20, 30, 40, 50, 100]}
fillna_knn_reg(
    df=df_prop,
    base=["taxvaluedollarcnt"],
    target="structuretaxvaluedollarcnt",
    tuning_params=parameters,
)
fillna_knn_reg(
    df=df_prop, base=["taxvaluedollarcnt"], target="taxamount", tuning_params=parameters
)
fillna_knn_reg(
    df=df_prop,
    base=["taxvaluedollarcnt"],
    target="landtaxvaluedollarcnt",
    tuning_params=parameters,
)

In [ ]:
tmp_col = df_prop.columns
tmp_list = []

for c in tmp_col:
    if df_prop[c].isna().sum() / df_prop.shape[0] > 0.0:
        tmp_list.append(c)

print("Number of features with missing values: ", len(tmp_list))
print(tmp_list)

Come si può notare restano solamente poche features con valori missing, si procede quindi:
- riempiendo 'numberofstories' con la sua moda
- rimuovendo la colonna 'censustractandblock' perché si assume che l'attributo 'rawcensustractandblock' contenga le stesse informazioni anche se in maniera meno elaborata.
- costruendo un predittore per 'calculatedfinishedsquarefeet' basato su 'bathroomcnt', 'bedroomcnt', 'structuretaxvaluedollarcnt'. Si ipotizza infatti che la dimensione totale dell'area abitabile sia dipendente da quanti bagni e camere da letto ci sono e da quante tasse sulla struttura dell'abitazione sono attribuite.
- costruendo un predittore per 'finishedfloor1squarefeet' basato sulle stesse features precedenti e sul numero di piani dell'edificio (numberofstories). Si assume che il numero di piani presenti possa essere utile per calcolare la features 'finishedfloor1squarefeet' (*Dimensioni della superficie abitativa finita al primo piano (ingresso) della casa*)

In [ ]:
plt.figure(figsize=(5, 5))
x_min = df_prop["numberofstories"].min()
x_max = df_prop["numberofstories"].max()
plt.xlim([x_min, x_max])
plt.xticks(np.arange(x_min, x_max + 1, 1))
df_prop["numberofstories"].hist()
plt.grid()
plt.show()

mode = float(df_prop["numberofstories"].mode()[0])
print("Moda: ", mode)
df_prop["numberofstories"] = df_prop["numberofstories"].fillna(mode)

In [150]:
df_prop.drop(columns="censustractandblock", inplace=True)

In [ ]:
fillna_knn_reg(
    df=df_prop,
    base=["bathroomcnt", "bedroomcnt", "structuretaxvaluedollarcnt"],
    target="calculatedfinishedsquarefeet",
    tuning_params=parameters,
)
fillna_knn_reg(
    df=df_prop,
    base=["bathroomcnt", "bedroomcnt", "structuretaxvaluedollarcnt", "numberofstories"],
    target="finishedfloor1squarefeet",
    tuning_params=parameters,
)

In [ ]:
tmp_col = df_prop.columns
tmp_list = []

for c in tmp_col:
    if df_prop[c].isna().sum() / df_prop.shape[0] > 0.0:
        tmp_list.append(c)

print("Number of missing features: ", len(tmp_list))

In [ ]:
print("Columns:")
print(df_prop.columns)
print("Shape: ", df_prop.shape)

<a id='custom_features'></a>
### Aggiunta di features custom potenzialmente utili

Vengono ora create ed aggiunte al dataset alcune features custom che intuitivamente potrebbero essere utili per la costruzione del modello finale e per spiegare meglio l'andamento dei dati

##### Features relative a proprietà dell'edificio

In [154]:
# Età dell'edificio al momento della vendita
df_prop["yearbuilt"] = pd.to_datetime(df_prop["yearbuilt"], format="%Y")
df_prop["assessmentyear"] = pd.to_datetime(df_prop["assessmentyear"], format="%Y")
df_prop["Life-until-selling"] = (
    df_prop["transactiondate"] - df_prop["yearbuilt"]
).dt.days

# Rapporto tra structure value e land area
df_prop["N-ValueProp"] = (
    df_prop["structuretaxvaluedollarcnt"] / df_prop["landtaxvaluedollarcnt"]
)

# Porzione di area vivibile
df_prop["N-LivingAreaProp"] = (
    df_prop["calculatedfinishedsquarefeet"] / df_prop["lotsizesquarefeet"]
)

# Quantità di spazio extra
df_prop["N-ExtraSpace"] = (
    df_prop["lotsizesquarefeet"] - df_prop["calculatedfinishedsquarefeet"]
)

# Features che indica se la proprietà ha garage, piscina, o ibromassaggio e AC
df_prop["N-GarPoolAC"] = (
    (df_prop["garagecarcnt"] > 0)
    & (df_prop["hashottuborspa"] > 0)
    & (df_prop["airconditioningtypeid"] != 5)
) * 1
df_prop["N-GarPoolAC"] = df_prop["N-GarPoolAC"].astype(bool)

In [155]:
# Rapporto tasse sulla casa su tasse totali per assesment year
df_prop["N-ValueRatio"] = df_prop["taxvaluedollarcnt"] / df_prop["taxamount"]

# Total Tax Score
df_prop["N-TaxScore"] = df_prop["taxvaluedollarcnt"] * df_prop["taxamount"]

##### Features relative alla posizione

In [156]:
# Numero di proprietà per zip code
zip_count = df_prop["regionidzip"].value_counts().to_dict()
df_prop["N-zip_count"] = df_prop["regionidzip"].map(zip_count)

# Numero di proprietà per città
city_count = df_prop["regionidcity"].value_counts().to_dict()
df_prop["N-city_count"] = df_prop["regionidcity"].map(city_count)

# Numero di proprietà per per paese
region_count = df_prop["regionidcounty"].value_counts().to_dict()
df_prop["N-county_count"] = df_prop["regionidcounty"].map(region_count)

##### Features che sono la semplificazione di altre features

In [157]:
# Indicatore se AC è presente o no
df_prop["N-ACInd"] = (df_prop["airconditioningtypeid"] != 5) * 1
df_prop["N-ACInd"] = df_prop["N-ACInd"].astype(bool)

# Indicatore se riscaldamento è presente o no
df_prop["N-HeatInd"] = (df_prop["heatingorsystemtypeid"] != 13) * 1
df_prop["N-HeatInd"] = df_prop["N-HeatInd"].astype(bool)

# Tipo di destinazione d'uso del terreno per il quale è suddiviso in zone l'immobile - prima erano 25 categorie, ora vengono compresse a 4
df_prop["N-PropType"] = df_prop["propertylandusetypeid"].replace(
    {
        31: "Mixed",
        46: "Other",
        47: "Mixed",
        246: "Mixed",
        247: "Mixed",
        248: "Mixed",
        260: "Home",
        261: "Home",
        262: "Home",
        263: "Home",
        264: "Home",
        265: "Home",
        266: "Home",
        267: "Home",
        268: "Home",
        269: "Not Built",
        270: "Home",
        271: "Home",
        273: "Home",
        274: "Other",
        275: "Home",
        276: "Home",
        279: "Home",
        290: "Not Built",
        291: "Not Built",
    }
)
df_prop.drop(columns="propertylandusetypeid", inplace=True)

##### Features custom relative a 'structuretaxvaluedollarcnt' perché viene considerata una specifica importante

In [158]:
# Media di structuretaxvaluedollarcnt per città
group = (
    df_prop.groupby("regionidcity")["structuretaxvaluedollarcnt"]
    .aggregate("mean")
    .to_dict()
)
df_prop["N-Avg-structuretaxvaluedollarcnt"] = df_prop["regionidcity"].map(
    group
)  # assegna 'regionidcity' alla media calcolata sopra e messa in un dizionario con chiave 'regionidcity'

# Discostamento del valore dalla media
df_prop["N-Dev-structuretaxvaluedollarcnt"] = (
    abs(
        (
            df_prop["structuretaxvaluedollarcnt"]
            - df_prop["N-Avg-structuretaxvaluedollarcnt"]
        )
    )
    / df_prop["N-Avg-structuretaxvaluedollarcnt"]
)

In [159]:
# Nel caso siano presenti valori "Infinito" nel dataset derivanti da divisioni per zero questi vengono posti a 0 in quanto semanticamente corretto
df_prop.replace([np.inf, -np.inf], 0, inplace=True)

In [ ]:
df_prop.info(max_cols=TOP_K_DISPLAY)
print("Shape: ", df_prop.shape)

<a id='features_selection'></a>
### Features importance, features selection e preparazione del dataset finale


Arrivati a questo punto il dataset non presenta più valori mancanti e intuitivamente sono stati risolti i problemi di multicollinearità tra le features date in input, evidenziati nelle analisi precedenti. Può essere quindi svolto il processo di features selection in modo da costruire modelli previsionali ancora più accurati

In [ ]:
cat_var_names = set(
    [
        "airconditioningtypeid",
        "heatingorsystemtypeid",
        "propertycountylandusecode",
        "N-PropType",
        "propertyzoningdesc",
        "regionidcity",
        "regionidcounty",
        "regionidneighborhood",
        "regionidzip",
        "fips",
        "rawcensustractandblock",
    ]
)

for c in cat_var_names:
    print(c, len(df_prop[c].unique()))

Si decide però di rimuovere alcune delle variabili categoriali con molti elementi distinti (nello specifico 'propertyzoningdesc', 'propertycountylandusecode', 'regionidneighborhood', 'regionidzip', 'regionidcity', 'rawcensustractandblock') in modo da non aumentare esponenzialmente la dimensione della matrice dopo il processo di OneHotEncoding (possibile anche problema di curse of dimensionality) ed innalzare il tempo richiesto per il training e il testing dei modelli.<br>
Si fa presente che questo processo di encoding è necessario per la corretta gestione di features categriali, altrimenti queste verrebbero interpretate in modo numerico o ordinale 

In [172]:
removed_cat = set(
    [
        "propertyzoningdesc",
        "propertycountylandusecode",
        "regionidneighborhood",
        "regionidzip",
        "regionidcity",
        "rawcensustractandblock",
    ]
)
one_hot_colmuns = list(cat_var_names.difference(removed_cat))


one_hot_enc = OneHotEncoder(sparse_output=False)
one_hot_enc.fit(df_prop[one_hot_colmuns])
one_hot_tranform_name = one_hot_enc.get_feature_names_out(one_hot_colmuns)

df_one_hot = pd.DataFrame(
    one_hot_enc.transform(df_prop[one_hot_colmuns]), columns=one_hot_tranform_name # type: ignore
)

df_prop_drop_cat = df_prop.drop(columns=list(cat_var_names))
df_prop_final = pd.concat(
    [df_prop_drop_cat.reset_index(), df_one_hot.reset_index()], axis=1
)
df_prop_final.drop(columns=["index"], inplace=True)


# Convert date to integer
df_prop_final["yearbuilt"] = df_prop_final["yearbuilt"].dt.year
df_prop_final["assessmentyear"] = df_prop_final["assessmentyear"].dt.year
to_float64_float32(df_prop_final)
to_int64_int32(df_prop_final)
del df_prop, df_one_hot

In [ ]:
df_prop_final.info(max_cols=TOP_K_DISPLAY)
print("Final shape: ", df_prop_final.shape)
df_prop_final.to_parquet(os.path.join(OUTPUT_DATA_DIR, "final_df_2016.parquet"))

In [61]:
X = df_prop_final.drop(columns=["parcelid", "logerror", "transactiondate"])
y = df_prop_final["logerror"]

X e y conterranno rispettivamente:
- le features da usare per la costruzione del modello
- i valori reali da utilizzare per il task di supervised learning

*Si fa notare che le variabili contengono solo dati di tipo float (o convertibili in float) perché unici tipi accettati dagli algoritmi di ML utilizzati*

Si procede nella sezione sottostante al tuning del parametro "n_estimators" per la costruzione di un modello sfruttante gradient boosting che una volta allenato verrà utilizzato per selezionare le features ritenute di maggior impatto dall'algoritmo<br>
Si fa notare inoltre che è stata scelta la libreria XGBoost per la sua elevata efficienza ed accuratezza

In [ ]:
tuning_params = {"n_estimators": [i for i in range(1, 26, 1)]}

X_train_80, X_test, y_train_80, y_test = train_test_split(X, y, test_size=0.20)

xgb_model = xgb.XGBRegressor()
xgb_grid = GridSearchCV(
    estimator=xgb_model,
    param_grid=tuning_params,
    cv=5,
    scoring="neg_root_mean_squared_error",
    verbose=0,
    n_jobs=-1,
)

print("Tuning XGBoost hyperparameters:")
xgb_grid.fit(X_train_80, y_train_80)
print("Best Score: {:.4f}".format(-xgb_grid.best_score_))
print("Best Params: ", xgb_grid.best_params_)

test_mse = root_mean_squared_error(y_true=y_test, y_pred=xgb_grid.predict(X_test))
print("MSE: {:.4f}".format(test_mse))

In [ ]:
plot_feature_importance(
    xgb_grid.best_estimator_.feature_importances_, X.columns, "XGBoost Model ", limit=40
)

*Si nota dal grafico riguardante la features importance che molte delle variabili custom aggiunte precedentemente hanno un peso rilevante nella costruzione del modello*

Viene ora scelto mediante il successivo metodo di feature ranking con recursive feature elimination usante cross-validation sets il presunto miglior sottinsieme di n features in modo da ridurre, come anticipato, la dimensionalità del dataframe ed i conseguenti problemi di multicollinearità e curse of dimentionality per ottimizzare maggiormente le previsioni.<br>
Si è deciso di impostare un numero minimo di 10 features in modo che nel modello finale non rimangano troppe poche variabili che di conseguenza non riuscirebbero a spiegare in modo sufficientemente accurato i dati.

In [ ]:
selector = RFECV(
    xgb_grid.best_estimator_,
    step=1,
    cv=5,
    scoring="neg_root_mean_squared_error",
    min_features_to_select=10,
    n_jobs=-1,
)
selector.fit(X, y)
selector.n_features_

Le features selezionate dall'algoritmo sono quindi 12, nello specifico:

In [ ]:
list_important_features = [
    x[0] for x in list(zip(X.columns, selector.support_)) if x[1]
]
print(list_important_features)

Viene creato e salvato offline il dataset contenete le features selezionate in modo che possa poi essere usato per la costruzione dei due modelli finali o per eventuali analisi future

In [ ]:
selected_important_X = X[list_important_features]
selected_important_X.shape

In [68]:
selected_important_X.to_parquet(OUTPUT_DATA_DIR + "final_df_2016_filtered.parquet")

<a id='mod_1'></a>
### Corstruzione del modello sfruttante Random Forest e tuning dei suoi parametri

In questa fase si è deciso di realizzare in modo più sofisticato il tuning degli parametri.
Nello specifico il metodo con cross validation sets cercherà la migliore combinazione di "n_estimators", "max_leaf_nodes" e "min_samples_leaf", 3 degli iperparametri principali per la costruzione del regressore sfruttante il metodo di ensembling Random Forest

In [69]:
X_train_80, X_test, y_train_80, y_test = train_test_split(
    selected_important_X, y, test_size=0.20
)

In [ ]:
tuning_params = {
    "n_estimators": [i for i in range(10, 81, 10)],
    "max_leaf_nodes": [
        10,
        30,
        50,
        100,
        200,
    ],  # Grow trees with max_leaf_nodes in best-first fashion
    "min_samples_leaf": [i for i in range(1, 5, 1)],
}  # The minimum number of samples required to be at a leaf node.

rf = RandomForestRegressor()
rf_model = GridSearchCV(
    estimator=rf,
    param_grid=tuning_params,
    cv=5,
    scoring="neg_root_mean_squared_error",
    verbose=0,
    n_jobs=-1,
)

print("Tuning Random Forest hyperparameters:")
rf_model.fit(X_train_80, y_train_80)
print("Best Score: {:.4f}".format(-rf_model.best_score_))
print("Best Params: ", rf_model.best_params_)

test_mse = root_mean_squared_error(y_true=y_test, y_pred=rf_model.predict(X_test))
print("MSE: {:.4f}".format(test_mse))

joblib.dump(rf_model, os.path.join(MODEL_DIR, "random_forest_model.pkl"))

Si ritiene che i valori di Mean Squared Error ottenuti, in particolare nel testing set, siano sufficientemente bassi e che di conseguenza il modello usante Random Forest sia abbastanza accurato. Inoltre utilizzando i cross validation sets si è limitata la possibilità di overfitting del predittore

In [ ]:
N_TESTS = 2
step = 2
offset = 10

stats = np.array([])
n_trees = [
    1 if i == 0 else i
    for i in range(0, rf_model.best_params_["n_estimators"] + offset, step)
]

for l in n_trees:
    y_preds = np.array([])

    for i in range(N_TESTS):
        Xs, ys = resample(X_train_80, y_train_80, n_samples=int(0.67 * len(y_train_80)))  # type: ignore

        rf = RandomForestRegressor(
            n_estimators=l,
            max_leaf_nodes=rf_model.best_params_["max_leaf_nodes"],
            min_samples_leaf=rf_model.best_params_["min_samples_leaf"],
            n_jobs=-1,
        )
        rf.fit(Xs, ys)

        y_pred = rf.predict(X_test)
        y_preds = np.column_stack([y_preds, y_pred]) if y_preds.size else y_pred

    dt_bias = (y_test - np.mean(y_preds, axis=1)) ** 2
    dt_variance = np.var(y_preds, axis=1)
    dt_error = (y_preds - y_test.reshape(-1, 1)) ** 2.0

    run_stats = np.array([dt_error.mean(), dt_bias.mean(), dt_variance.mean()])

    stats = np.column_stack([stats, run_stats]) if stats.size else run_stats


fig, ax = plt.subplots(figsize=(8, 8))
ax.set_title("Bias-Variance Decomposition Analysis - Random Forest")
ax.plot(n_trees, stats[0, :], "o:", label="Error")
ax.plot(n_trees, stats[1, :], "o:", label="Bias$^2$")
ax.plot(n_trees, stats[2, :], "o:", label="Variance")
ax.set_xlabel("Number of Trees")
ax.grid()
ax.legend()

print("Error/Bias/Variance at the last iteration:", stats[:, -1])

Come si può notare dal grafico *numero alberi - errore totale*, l'errore diminuisce (e lo score migliora) all'aumetare del numero di stimatori utilizzati. In particolare si evince, come da intuizione teorica, che essendo il predittore basato su foresta di alberi ogniuno di questi per definizione si cerca sia fully grown, quindi con bassa distorsione e alta varianza, e che quest'ultima varrà poi ridotta con l'ensembing. In questo specifico caso è evidenziato come si riesca a ridurre la varianza in modo significativo già usando solamente 4 stimatori.<br>
Si ricorda poi che la formula di decomposizione dell'errore totale è:

<a id='mod_2'></a>
### Corstruzione del modello sfruttante Gradient Boosting e tuning dei suoi parametri

Anche per la costruzione di questo modello si è deciso di realizzare più sofisticatamente il tuning degli parametri.
Nello specifico il metodo con cross validation sets cercherà la migliore combinazione di "n_estimators", "max_leaves" e "learning_rate", 3 degli iperparametri principali per la costruzione del regressore sfruttante il metodo Gradient Boosting

In [75]:
X_train_80, X_test, y_train_80, y_test = train_test_split(
    selected_important_X, y, test_size=0.20
)

In [ ]:
tuning_params = {
    "n_estimators": [i for i in range(10, 101, 10)],
    "max_leaves": [
        2,
        5,
        10,
        50,
        100,
        200,
    ],  # Maximum number of leaves; 0 indicates no limit
    "learning_rate": [0.1, 0.2, 0.3, 0.4],
}  # Boosting learning rate (xgb’s “eta”).
# Step size shrinkage used in update to prevents overfitting. The value must be between 0 and 1. Default is 0.3.

xgb_m = xgb.XGBRegressor()
xgb_model = GridSearchCV(
    estimator=xgb_m,
    param_grid=tuning_params,
    cv=5,
    scoring="neg_root_mean_squared_error",
    verbose=0,
    n_jobs=-1,
)

print("Tuning XGBoost hyperparameters:")
xgb_model.fit(X_train_80, y_train_80)
print("Best Score: {:.4f}".format(-xgb_model.best_score_))
print("Best Params: ", xgb_model.best_params_)

test_mse = root_mean_squared_error(y_true=y_test, y_pred=xgb_model.predict(X_test))
print("MSE: {:.4f}".format(test_mse))

joblib.dump(xgb_model, os.path.join(MODEL_DIR, "xgboost_model.pkl"))

Anche in questo caso si ritiene che i valori di Mean Squared Error ottenuti, in particolare nel testing set, siano sufficientemente bassi e che di conseguenza il modello basato su Gradient Boosting sia abbastanza accurato. Inoltre utilizzando i cross validation sets si è limitata la possibilità di overfitting nel regressore

In [ ]:
N_TESTS = 2
step = 2
offset = 10

stats = np.array([])
n_trees = [
    1 if i == 0 else i
    for i in range(0, xgb_model.best_params_["n_estimators"] + offset, step)
]

for l in n_trees:
    y_preds = np.array([])

    for i in range(N_TESTS):
        Xs, ys = resample(X_train_80, y_train_80, n_samples=int(0.67 * len(y_train_80)))  # type: ignore

        xgb_m = xgb.XGBRegressor(
            n_estimators=l,
            learning_rate=xgb_model.best_params_["learning_rate"],
            max_leaves=xgb_model.best_params_["max_leaves"],
            n_jobs=-1,
        )
        xgb_m.fit(Xs, ys)

        y_pred = xgb_m.predict(X_test)
        y_preds = np.column_stack([y_preds, y_pred]) if y_preds.size else y_pred

    dt_bias = (y_test - np.mean(y_preds, axis=1)) ** 2
    dt_variance = np.var(y_preds, axis=1)
    dt_error = (y_preds - y_test.reshape(-1, 1)) ** 2.0

    run_stats = np.array([dt_error.mean(), dt_bias.mean(), dt_variance.mean()])

    stats = np.column_stack([stats, run_stats]) if stats.size else run_stats


fig, ax = plt.subplots(figsize=(8, 8))
ax.set_title("Bias-Variance Decomposition Analysis - XGBoost")
ax.plot(n_trees, stats[0, :], "o:", label="Error")
ax.plot(n_trees, stats[1, :], "o:", label="Bias$^2$")
ax.plot(n_trees, stats[2, :], "o:", label="Variance")
ax.set_xlabel("Number of Trees")
ax.grid()
ax.legend()

print("Error/Bias/Variance at the last iteration:", stats[:, -1])

Come si può notare dal grafico, anche in questo caso l'errore dimiuisce (e lo score migliora) all'aumetare del numero di stimatori utilizzato. In particolare si evince, come da intuizione teorica, che essendo il predittore basato su boosting di alberi, ogniuno di questi per definizione si cerca sia di dimensioni ridotte, quindi con bassa varianza ma alta distorsione, e che quest'ultima varrà varrà poi ridotta con l'ensembing. 
In questo specifico caso la decomposizione mostra una varianza quasi nulla ed un errore che descresce esponenzialmente al diminuire della distorzione, che si riesce inoltre a ridurre in modo significativo già usando solamente 10 stimatori.

<a id='comparison_and_analisys'></a>
### Comparazione e analisi dei modelli

Ultimata la costruzione dei modelli statistici finali si porcede ora ad analizzarne più approfonditamente il comportamento, in particolare fecendo un focus su alcuni degli edifici dove il logerror dato è il peggiore ed altri dove è il migliore.

Vengono quindi estratti gli edifici con log error compreso tra il valore più estremo e il valore più estremo meno 1 (worst items) e dieci tra le costruzioni dove il log error è zero (best items)

In [ ]:
sns.displot(df_prop_final, x="logerror", height=6, aspect=16 / 9);

La variabile 'logerror' segue una distribuzione normale

In [96]:
offset = 1
max_error_pos = df_prop_final["logerror"].max() - offset
max_error_neg = df_prop_final["logerror"].min() + offset

worst_items = pd.concat(
    [
        df_prop_final[df_prop_final["logerror"] >= max_error_pos],
        df_prop_final[df_prop_final["logerror"] <= max_error_neg],
    ]
)
best_items = df_prop_final[df_prop_final["logerror"] == 0].head(10)

In [ ]:
worst_items_X = selected_important_X.iloc[worst_items.index]
worst_items_X

In [ ]:
best_items_X = selected_important_X.iloc[best_items.index]
best_items_X

In [ ]:
plt.figure(figsize=(8, 6))
plt.bar(
    range(len(worst_items)),
    worst_items["logerror"],
    color=["#9dd866"],
    label="Target value",
)
plt.bar(
    range(len(worst_items)),
    rf_model.predict(worst_items_X),
    color=["#ffa056"],
    label="RF value",
)
plt.bar(
    range(len(worst_items)),
    xgb_model.predict(worst_items_X),
    color=["#0b84a5"],
    label="XGB value",
)
plt.legend()
plt.title("Log-errors comparisons - Worst items");

Dal bar chart relativo alle previsioni su item problematici si può notare che le stime dei due modelli finali creati sono molto distanti dal valore target calcolato dallo stimatore Zestimate. Probabilmente questo è dovuto alla presenza di molti valori missing o outliers, trattati con tecniche di analisi, recupero e correzione differenti da quelle usate dal team di Zestimate.
A priori però si ipotizza che gli item con logerror in valore assoluto elevato possano presentare anomalie o valori non standard per alcune features, in quanto proprio lo stimatore della competizione restituisce dei valori predetti abbastanza distanti dal log(SalePrice) reale.

Per verificare queste ipotesi si pensa sia necessario contattare degli esperti in ambito Data Science, il team di Zestimate e dei tecnici immobiliari.

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(
    range(len(best_items)),
    best_items["logerror"],
    color="#9dd866",
    label="Target value",
)
plt.plot(
    range(len(best_items)),
    rf_model.predict(best_items_X),
    color="#ffa056",
    label="RF value",
)
plt.plot(
    range(len(best_items)),
    xgb_model.predict(best_items_X),
    color="#0b84a5",
    label="XGB value",
)
plt.legend()
plt.title("Log-errors comparisons - Best Items");

Nel line chart inerente alle previsioni su item il cui log error è nullo invece anche i due modelli creati restituiscono valori soddisfacienti: infatti, il discostamento rispetto al valore target di Zestimate è nell'ordine dei centesini

In generale, anche nei grafici relativi a best items e worst items, non si notano grosse differenze in termini di accuratezza tra il modello con Random Forest e quello con Gradient Boosting, anche se il valore di MSE ottenuto nel testing set del primo modello e leggermente inferiore rispetto a quello restituito dal secondo.

<a id='final'></a>
### Considerazioni finali

Vengono quindi brevemente ricapitolati i vari passaggi svolti:
- Analisi dei dataset, features engineering e gestione dei missing value
- Features selection e costruzione dei modelli
- Analisi e comparazione dei modelli ottenuti

Concludendo, grazie a questo task si sono potuto testare su un dataset reale, esteso e complesso, le performance di alcuni metodi di Machine Learning come Random Forest e Gradient Boosting, entrambi sfruttanti alberi di decisione ma utilizzanti intuizioni diverse.<br>
Tuttavia, come anticipato in precedenza, non si dispone di sufficienti elementi per preferire un modello rispetto all'altro in quanto entrambi ritornano previsioni accurate ed in modo efficiente e il discostamento nel valore della metrica di valutazione per il testing set è molto bassa.
Generalmente, i predittori che utilizzano la tecnica Gradient Boosting performano meglio di quelli che sfruttano Random Forest per definizione in quanto hanno come obiettivo la minimizzazione di una funzione di perdita e la caratteristica di costruire additivamente i vari alberi, anche se sono più proni all'overfitting.<br>
Probabilmente, con una conoscenza più approfondita dei dati, del loro significato, del loro andamento reale e con l'aggiunta di informazioni esterne al dataset è possibile arrivare ad ottenere risultati ancora più precisi ed alla costruzione del modello di regressione ottimo.

##### *Tabella riassuntiva contenente i dati delle valutazioni*
| Set type | Model | MSE Score |
| --- | --- | --- |
| Validation Sets | Random Forest | 0.0258 |
| Validation Sets | Gradient Boosting | 0.0259 |
| Testing Set | Random Forest | 0.0254 |
| Testing Set | Gradient Boosting | 0.0257 |